Implementation of SMPA with Gradient descent and PyTorch here only y moves and with Pchip it is pretty good and beating industry standards

In [ ]:
class BSpline(nn.Module):
    """Differentiable B-spline implementation in PyTorch"""
    def __init__(self, knots, degree=3):
        super(BSpline, self).__init__()
        self.knots = knots  # Knot vector
        self.degree = degree

    def basis(self, t, i, k):
        """Recursive De Boor basis function"""
        if k == 0:
            return torch.where((self.knots[i] <= t) & (t < self.knots[i+1]),
                              torch.ones_like(t), torch.zeros_like(t))

        d1 = (self.knots[i+k] - self.knots[i])
        d2 = (self.knots[i+k+1] - self.knots[i+1])

        # Avoid division by zero
        c1 = torch.zeros_like(t)
        mask1 = d1 > 0
        c1[mask1] = (t[mask1] - self.knots[i]) / d1 * self.basis(t[mask1], i, k-1)

        c2 = torch.zeros_like(t)
        mask2 = d2 > 0
        c2[mask2] = (self.knots[i+k+1] - t[mask2]) / d2 * self.basis(t[mask2], i+1, k-1)

        return c1 + c2

    def forward(self, t, control_points):
        n = len(control_points) - 1
        result = torch.zeros_like(t)

        for i in range(n + 1):
            result += control_points[i] * self.basis(t, i, self.degree)

        return result

class TorchCubicSpline(nn.Module):
    """Differentiable cubic spline implementation using PyTorch"""
    def __init__(self, x, y):
        super(TorchCubicSpline, self).__init__()
        self.x = x  # Control point x-coordinates (fixed)
        self.y = nn.Parameter(y)  # Control point y-coordinates (learnable)

        # Compute knot vector for cubic B-spline
        n = len(x)
        self.knots = torch.cat([
            x[0].repeat(3),
            x[1:-1],
            x[-1].repeat(3)
        ])

        self.bspline = BSpline(self.knots, degree=3)

    def forward(self, t):
        # Ensure t is clamped to domain
        t_clamped = torch.clamp(t, self.x[0], self.x[-1])

        # Evaluate B-spline
        return self.bspline(t_clamped, self.y)

class DifferentiablePchip(nn.Module):
    """Fully differentiable PCHIP (monotonic) spline in PyTorch"""
    def __init__(self, x, y):
        super(DifferentiablePchip, self).__init__()
        self.x = x  # Control point x-coordinates (fixed)
        self.y = nn.Parameter(y)  # Control point y-coordinates (learnable)
        self.n = len(x) - 1

    def forward(self, t):
        # Find the segment for each input t
        idx = torch.searchsorted(self.x, t) - 1
        idx = torch.clamp(idx, 0, self.n - 1)

        # Get x and y values for the segment
        x0 = self.x[idx]
        x1 = self.x[idx + 1]
        y0 = torch.gather(self.y, 0, idx)
        y1 = torch.gather(self.y, 0, idx + 1)

        # Normalize t within segment
        t_norm = (t - x0) / (x1 - x0)

        # Calculate slopes
        dy = self.y[1:] - self.y[:-1]
        dx = self.x[1:] - self.x[:-1]
        slopes = dy / dx

        # Enforce monotonicity by adjusting derivatives
        # Simplified PCHIP algorithm
        d = torch.zeros_like(self.y)
        for i in range(1, len(self.y)-1):
            if slopes[i-1] * slopes[i] > 0:
                # Same sign, use harmonic mean for monotonicity
                w1 = 2*dx[i] + dx[i-1]
                w2 = dx[i] + 2*dx[i-1]
                d[i] = (w1 + w2) / (w1/slopes[i-1] + w2/slopes[i])
            else:
                # Different signs or one is zero
                d[i] = 0

        # Handle endpoints
        d[0] = slopes[0]
        d[-1] = slopes[-1]

        # Get derivatives for segment
        d0 = torch.gather(d, 0, idx)
        d1 = torch.gather(d, 0, idx + 1)

        # Hermite basis functions
        h00 = 2*t_norm**3 - 3*t_norm**2 + 1
        h10 = t_norm**3 - 2*t_norm**2 + t_norm
        h01 = -2*t_norm**3 + 3*t_norm**2
        h11 = t_norm**3 - t_norm**2

        # Apply cubic Hermite interpolation with monotonic derivatives
        dx_segment = x1 - x0
        result = (h00 * y0 + h10 * dx_segment * d0 +
                 h01 * y1 + h11 * dx_segment * d1)

        return result

class PyTorchGradientSMPA(BaseEstimator, ClassifierMixin):
    def __init__(self, learning_rate=0.05, epochs=100, random_state=7, verbose=False,
                 lambda_reg=0.0001, patience=10, decay_factor=0.9, min_learning_rate=1e-6,
                 n_control_points=6, smoothing_factor=0.0001, spline_type='cubic',
                 device=None, track_history=False, optimizer_type='adam', scheduler_type='reduce_on_plateau'):
        self.learning_rate = learning_rate
        self.initial_learning_rate = learning_rate
        self.epochs = epochs
        self.random_state = random_state
        self.verbose = verbose
        self.lambda_reg = lambda_reg
        self.patience = patience
        self.decay_factor = decay_factor
        self.min_learning_rate = min_learning_rate
        self.n_control_points = n_control_points
        self.smoothing_factor = smoothing_factor
        self.spline_type = spline_type
        self.device = device if device is not None else ('cuda' if torch.cuda.is_available() else 'cpu')
        self.track_history = track_history
        self.optimizer_type = optimizer_type
        self.scheduler_type = scheduler_type

        if spline_type not in ['cubic', 'pchip']:
            raise ValueError("spline_type must be 'cubic' or 'pchip'")

        torch.manual_seed(random_state)
        np.random.seed(random_state)

    def _to_tensor(self, data, dtype=torch.float32):
        if isinstance(data, torch.Tensor):
            return data.to(self.device).to(dtype)
        elif isinstance(data, pd.Series):
            # Convert pandas Series to numpy array first
            return torch.tensor(data.values, dtype=dtype, device=self.device)
        elif isinstance(data, pd.DataFrame):
            # Convert pandas DataFrame to numpy array first
            return torch.tensor(data.values, dtype=dtype, device=self.device)
        elif isinstance(data, np.ndarray):
            # Handle numpy arrays
            return torch.tensor(data, dtype=dtype, device=self.device)
        else:
            # For any other array-like objects
            try:
                return torch.tensor(np.array(data), dtype=dtype, device=self.device)
            except:
                raise ValueError(f"Cannot convert {type(data)} to tensor: {data}")

    def _calculate_class_means(self, X, y):
        mask_1 = y == 1
        self.m1 = torch.mean(X[mask_1], dim=0)
        self.m0 = torch.mean(X[~mask_1], dim=0)

    def _initialize_control_points(self, X):
        n_features = X.shape[1] - 1  # All features except the last one
        self.spline_models = nn.ModuleList()

        for i in range(n_features):
            x_min, x_max = X[:, i].min().item(), X[:, i].max().item()
            control_x = torch.linspace(x_min, x_max, self.n_control_points, device=self.device)

            # Initialize y values around the mean of the target feature
            y_min, y_max = X[:, -1].min().item(), X[:, -1].max().item()
            y_mid = (self.m0[-1] + self.m1[-1]) / 2
            y_range = y_max - y_min
            control_y = torch.empty(self.n_control_points, device=self.device).uniform_(
                y_mid - y_range * 0.05, y_mid + y_range * 0.05
            )

            if self.spline_type == 'cubic':
                spline = TorchCubicSpline(control_x, control_y).to(self.device)
            else:
                spline = DifferentiablePchip(control_x, control_y).to(self.device)

            self.spline_models.append(spline)

        # Store initial control points for plotting or history
        self.initial_control_points = [(m.x.clone(), m.y.clone()) for m in self.spline_models]

    def _calculate_displacement(self, X):
        """Calculate total displacement as sum of spline contributions minus target feature"""
        total_spline = torch.zeros(X.shape[0], device=self.device)
        for i, spline in enumerate(self.spline_models):
            total_spline += spline(X[:, i])
        return X[:, -1] - total_spline

    def _update_pseudo_labels(self, X, y):
        m1_reshaped = self.m1.unsqueeze(0)
        m0_reshaped = self.m0.unsqueeze(0)
        m1_displacement = self._calculate_displacement(m1_reshaped)[0]

        self.class_1_pseudo = 1 if m1_displacement > 0 else -1
        self.class_0_pseudo = -self.class_1_pseudo

        return torch.where(y == 1,
                         torch.tensor(self.class_1_pseudo, device=self.device),
                         torch.tensor(self.class_0_pseudo, device=self.device))

    def _create_optimizer_and_scheduler(self):
        params = [p for spline in self.spline_models for p in spline.parameters()]
        if self.optimizer_type.lower() == 'adam':
            optimizer = torch.optim.Adam(params, lr=self.initial_learning_rate)
        else:
            optimizer = torch.optim.SGD(params, lr=self.initial_learning_rate)

        if self.scheduler_type.lower() == 'reduce_on_plateau':
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=self.decay_factor,
                patience=self.patience, min_lr=self.min_learning_rate, verbose=False
            )
        elif self.scheduler_type.lower() == 'step':
            scheduler = torch.optim.lr_scheduler.StepLR(
                optimizer, step_size=self.patience, gamma=self.decay_factor
            )
        else:
            scheduler = None

        return optimizer, scheduler

    def fit(self, X, y):
        if not set(np.unique(y)).issubset({0, 1}):
            raise ValueError("Labels must be 0 and 1")
        if X.shape[1] < 2:
            raise ValueError("At least 2 features required (one for spline, one as target)")

        X_tensor = self._to_tensor(X)
        y_tensor = self._to_tensor(y, dtype=torch.long)

        self._calculate_class_means(X_tensor, y_tensor)
        self._initialize_control_points(X_tensor)

        optimizer, scheduler = self._create_optimizer_and_scheduler()

        best_error = float('inf')
        best_control_ys = [spline.y.clone() for spline in self.spline_models]
        best_class_1_pseudo = None

        if self.track_history:
            self.error_history_ = []
            self.control_point_history = [self.initial_control_points]

        for epoch in range(self.epochs):
            pseudo_labels = self._update_pseudo_labels(X_tensor, y_tensor)
            displacements = self._calculate_displacement(X_tensor)

            errors = displacements * pseudo_labels <= 0
            error_count = errors.sum().item()

            if self.verbose and epoch % 5 == 0:
                current_lr = optimizer.param_groups[0]['lr']
                print(f"Epoch {epoch}: Errors = {error_count}, LR = {current_lr:.6f}")

            if error_count < best_error:
                best_error = error_count
                best_control_ys = [spline.y.clone() for spline in self.spline_models]
                best_class_1_pseudo = self.class_1_pseudo
                self.best_epoch = epoch

                if error_count == 0 and epoch > 10:
                    if self.verbose:
                        print(f"Perfect separation achieved at epoch {epoch}")
                    break

            if self.track_history:
                self.error_history_.append(error_count)
                self.control_point_history.append(
                    [(s.x.clone().cpu().numpy(), s.y.clone().detach().cpu().numpy())
                     for s in self.spline_models]
                )

            if error_count == 0:
                continue

            error_indices = torch.where(errors)[0]
            X_err = X_tensor[error_indices]
            y_err = y_tensor[error_indices]

            ti = torch.where(y_err == 1,
                           torch.tensor(1, device=self.device),
                           torch.tensor(-1, device=self.device))

            spline_values = torch.zeros(X_err.shape[0], device=self.device)
            for i, spline in enumerate(self.spline_models):
                spline_values += spline(X_err[:, i])

            loss = torch.mean(torch.relu(1.0 - ti * self.class_1_pseudo * (X_err[:, -1] - spline_values)))

            if self.lambda_reg > 0:
                smoothness_penalty = 0
                for spline in self.spline_models:
                    y_diff = spline.y[1:] - spline.y[:-1]
                    x_diff = spline.x[1:] - spline.x[:-1]
                    smoothness_penalty += torch.mean((y_diff / x_diff)**2)
                loss += self.lambda_reg * smoothness_penalty

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if scheduler is not None:
                if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                    scheduler.step(error_count)
                else:
                    scheduler.step()

                if optimizer.param_groups[0]['lr'] <= self.min_learning_rate:
                    if self.verbose:
                        print(f"Minimum learning rate reached at epoch {epoch}")
                    break

        for spline, best_y in zip(self.spline_models, best_control_ys):
            spline.y.data = best_y
        self.class_1_pseudo = best_class_1_pseudo

        return self

    def predict(self, X):
        X_tensor = self._to_tensor(X)
        displacements = self._calculate_displacement(X_tensor)
        predictions = torch.where(displacements > 0,
                                torch.tensor(1 if self.class_1_pseudo > 0 else 0, device=self.device),
                                torch.tensor(0 if self.class_1_pseudo > 0 else 1, device=self.device))
        return predictions.cpu().numpy()

    def predict_proba(self, X):
        X_tensor = self._to_tensor(X)
        displacements = self._calculate_displacement(X_tensor)
        raw_probs = 1 / (1 + torch.exp(-displacements * self.class_1_pseudo * 0.5))

        if self.class_1_pseudo > 0:
            probs = torch.column_stack([1 - raw_probs, raw_probs])
        else:
            probs = torch.column_stack([raw_probs, 1 - raw_probs])

        return probs.cpu().numpy()

    def plot_convergence(self, figsize=(10, 4)):
        if not self.track_history or not hasattr(self, 'error_history_'):
            print("Convergence plotting requires track_history=True and a fitted model.")
            return None

        fig, ax = plt.subplots(figsize=figsize)
        ax.plot(self.error_history_, 'b-', label='Errors')
        ax.set_title('Error Convergence')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Number of Errors')
        ax.grid(True)
        ax.legend()
        plt.tight_layout()
        return fig